## CORRESPONDÊNCIA ENTRE DESCRIÇÃO E HABILIDADES

In [94]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # mesmo que não use GPU, garante que o cache está limpo

In [95]:
from sentence_transformers import SentenceTransformer, models
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
from datetime import date
from math import log
import pyodbc
import warnings
import re
from unidecode import unidecode
warnings.filterwarnings('ignore')
from openai import AzureOpenAI
from sklearn.metrics.pairwise import cosine_similarity

In [96]:
from unidecode import unidecode

In [129]:
client = AzureOpenAI(
    api_key="ec1da5ea6acf4e96b545076ee750762a",
    api_version="2024-05-01-preview",
    azure_endpoint="https://snc-openai-ia-eu2-prd.openai.azure.com/",
)

data_match = date.today().strftime("%Y-%m-%d")
diretorio = 'C:/Users/rodrigo.michel/Documents/Dinamica_emprego/Habilidades/vagas_habilidades/'

server = 'SRVVPRDGPA01.senac.br'
database = 'DBMercadoTrabalho'
username = 'SRV_DMT'
password = 'lEJA6EiFabpcnR1f2qzLRXnb'
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)
conn = pyodbc.connect(conn_str)

### CARREGANDO A BASE DE VAGAS

In [5]:
query = "SELECT source, identifier, uuid, title, title_standardized, description  FROM job_vacancy.job_vacancy_post where source = 'BNE' ORDER BY NEWID()"

In [6]:
vagas = pd.read_sql(query, conn)
vagas.head()

,source,identifier,uuid,title,title_standardized,description
0,BNE,4517464,485618D1-3592-470E-9810-A5036401024C,Cargo/Função: supervisor de vendas,cargo funcao supervisor vendas,None
1,BNE,4047558,A68BA07D-2515-494E-9B08-C2D509355E92,pedreiro,pedreiro,Descrição Geral\nEmpresa localizada na cidade ...
2,BNE,4196822,4A13075A-B84A-4D8D-9ED4-4109D986634D,desenvolvedor,desenvolvedor,Descrição Geral\nEmpresa localizada na cidade ...
3,BNE,4152591,3ADD8F96-274C-4AC2-869E-A36CD835F553,estoquista,estoquista,Descrição Geral\nEmpresa localizada na cidade ...
4,BNE,4225298,AF23DA13-6E63-4D01-8A77-D1950DBCBEC1,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...


In [7]:
vagas = vagas.drop_duplicates()
vagas

,source,identifier,uuid,title,title_standardized,description
0,BNE,4517464,485618D1-3592-470E-9810-A5036401024C,Cargo/Função: supervisor de vendas,cargo funcao supervisor vendas,None
1,BNE,4047558,A68BA07D-2515-494E-9B08-C2D509355E92,pedreiro,pedreiro,Descrição Geral\nEmpresa localizada na cidade ...
2,BNE,4196822,4A13075A-B84A-4D8D-9ED4-4109D986634D,desenvolvedor,desenvolvedor,Descrição Geral\nEmpresa localizada na cidade ...
3,BNE,4152591,3ADD8F96-274C-4AC2-869E-A36CD835F553,estoquista,estoquista,Descrição Geral\nEmpresa localizada na cidade ...
4,BNE,4225298,AF23DA13-6E63-4D01-8A77-D1950DBCBEC1,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...
...,...,...,...,...,...,...
566060,BNE,4383863,A10EDA8C-BF4A-49DD-87C4-1950BF95A6F4,estilista,estilista,Descrição Geral\nEmpresa localizada na cidade ...
566061,BNE,4082387,0935370C-414D-4CC4-BA0F-6AA8088573D9,recepcionista,recepcionista,Descrição Geral\nEmpresa localizada na cidade ...
566062,BNE,4120573,D710A67D-DCC6-443F-8841-652E8ACBC342,promotor de vendas,promotor vendas,Descrição Geral\nEmpresa localizada na cidade ...
566063,BNE,3961403,461C0F13-CD1F-4C1E-9AF2-E4660DDFAF34,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...


In [8]:
vagas['description'].head(10)

0                                                 None
1    Descrição Geral\nEmpresa localizada na cidade ...
2    Descrição Geral\nEmpresa localizada na cidade ...
3    Descrição Geral\nEmpresa localizada na cidade ...
4    Descrição Geral\nEmpresa localizada na cidade ...
5                                                 None
6    Descrição Geral\nEmpresa localizada na cidade ...
7                                                 None
8    Descrição Geral\nEmpresa localizada na cidade ...
9    Descrição Geral\nEmpresa localizada na cidade ...
Name: description, dtype: object

In [9]:
vagas = vagas.dropna(subset=['description'])
vagas

,source,identifier,uuid,title,title_standardized,description
1,BNE,4047558,A68BA07D-2515-494E-9B08-C2D509355E92,pedreiro,pedreiro,Descrição Geral\nEmpresa localizada na cidade ...
2,BNE,4196822,4A13075A-B84A-4D8D-9ED4-4109D986634D,desenvolvedor,desenvolvedor,Descrição Geral\nEmpresa localizada na cidade ...
3,BNE,4152591,3ADD8F96-274C-4AC2-869E-A36CD835F553,estoquista,estoquista,Descrição Geral\nEmpresa localizada na cidade ...
4,BNE,4225298,AF23DA13-6E63-4D01-8A77-D1950DBCBEC1,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...
6,BNE,4307413,C5F8FA3F-F559-4B36-8661-A9EDDB5E8D68,assistente de logística,assistente logistica,Descrição Geral\nEmpresa localizada na cidade ...
...,...,...,...,...,...,...
566059,BNE,3942948,FFA43E9C-3D4A-4D2A-8334-C2FFAF92FF73,consultor de vendas,consultor vendas,Descrição Geral\nEmpresa localizada na cidade ...
566060,BNE,4383863,A10EDA8C-BF4A-49DD-87C4-1950BF95A6F4,estilista,estilista,Descrição Geral\nEmpresa localizada na cidade ...
566061,BNE,4082387,0935370C-414D-4CC4-BA0F-6AA8088573D9,recepcionista,recepcionista,Descrição Geral\nEmpresa localizada na cidade ...
566062,BNE,4120573,D710A67D-DCC6-443F-8841-652E8ACBC342,promotor de vendas,promotor vendas,Descrição Geral\nEmpresa localizada na cidade ...


In [54]:
vagas

,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade
1,BNE,4047558,A68BA07D-2515-494E-9B08-C2D509355E92,pedreiro,pedreiro,Descrição Geral\nEmpresa localizada na cidade ...,"pedreiro.,construções, reformas e reparos. ens...",construcao
2,BNE,4196822,4A13075A-B84A-4D8D-9ED4-4109D986634D,desenvolvedor,desenvolvedor,Descrição Geral\nEmpresa localizada na cidade ...,"desenvolvedor.,a vaga ecessita experiência com...",informatica
3,BNE,4152591,3ADD8F96-274C-4AC2-869E-A36CD835F553,estoquista,estoquista,Descrição Geral\nEmpresa localizada na cidade ...,"estoquista.,vaga aberta para estoquista n. sho...",logistica
4,BNE,4225298,AF23DA13-6E63-4D01-8A77-D1950DBCBEC1,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor.,atendimento ao público, vendas e pós...",comercio
6,BNE,4307413,C5F8FA3F-F559-4B36-8661-A9EDDB5E8D68,assistente de logística,assistente logistica,Descrição Geral\nEmpresa localizada na cidade ...,"assistente de logística.,apoio aos motoristas ...",logistica
...,...,...,...,...,...,...,...,...
566059,BNE,3942948,FFA43E9C-3D4A-4D2A-8334-C2FFAF92FF73,consultor de vendas,consultor vendas,Descrição Geral\nEmpresa localizada na cidade ...,"consultor de vendas., proatividade. boa comuni...",comercio
566060,BNE,4383863,A10EDA8C-BF4A-49DD-87C4-1950BF95A6F4,estilista,estilista,Descrição Geral\nEmpresa localizada na cidade ...,"estilista.,estamos contratando estilista para ...",marketing
566061,BNE,4082387,0935370C-414D-4CC4-BA0F-6AA8088573D9,recepcionista,recepcionista,Descrição Geral\nEmpresa localizada na cidade ...,"recepcionista.,como recepcionista você deverá ...",administrativo
566062,BNE,4120573,D710A67D-DCC6-443F-8841-652E8ACBC342,promotor de vendas,promotor vendas,Descrição Geral\nEmpresa localizada na cidade ...,"promotor de vendas.,oportunidade imperdível: p...",comercio


In [55]:
vagas.to_csv(diretorio + 'vagas.csv', sep=";", encoding='utf-8-sig')

### LIDANDO COM O TEXTO, LIMPANDO E ANALISANDO

In [10]:
import re
from unidecode import unidecode

def processar_descricao(texto):
    if not isinstance(texto, str):
        return {"descricao_limpa": "", "ramo_atividade": None}

    texto_original = texto
    texto_ascii = unidecode(texto_original)

    # Corrigir quebras de linha (literal ou real)
    texto_ascii = texto_ascii.replace("\\n", "\n").replace("\\r", "\n").replace("\r", "\n")
    
    # Extrair ramo após "do ramo ... contrata"
    ramo_match = re.search(r'do ramo\s+([^\n,]+)', texto_ascii, flags=re.IGNORECASE)
    ramo = ramo_match.group(1).strip() if ramo_match else None

    # Remover tudo até e incluindo "contrata"
    texto_limpo = re.sub(r'^.*?contrata', '', texto_original, flags=re.IGNORECASE)

    # Trocar quebras por espaço
    texto_limpo = texto_limpo.replace("\\n", " ").replace("\n", " ").replace("\r", " ")

    # Corrigir "nnossa", "natendimento", etc.
    texto_limpo = re.sub(r'\b[nN](?=\w)', '', texto_limpo)

    # Limpeza adicional
    texto_limpo = re.sub(r'Tipo de V[ií]nculo.*?(?=\n|$)', '', texto_limpo, flags=re.IGNORECASE)
    texto_limpo = re.sub(r'Atribuições?\s*:?[\n]?', '', texto_limpo, flags=re.IGNORECASE)
    texto_limpo = re.sub(r'Requisitos?\s*:?[\n]?', '', texto_limpo, flags=re.IGNORECASE)
    texto_limpo = re.sub(r'Escolaridade.*?(?=\n|$)', '', texto_limpo, flags=re.IGNORECASE)
    texto_limpo = re.sub(r'Remuneraç[aã]o.*?(?=\n|$)', '', texto_limpo, flags=re.IGNORECASE)
    texto_limpo = re.sub(r'#\w+', '', texto_limpo)
    texto_limpo = re.sub(r'[^\w\s.,:\n-]', '', texto_limpo)
    texto_limpo = re.sub(r'\s*-\s*', ' ', texto_limpo)
    texto_limpo = re.sub(r'\s{2,}', ' ', texto_limpo)

    texto_limpo = texto_limpo.strip().lower()

    return {
        "descricao_limpa": texto_limpo,
        "ramo_atividade": ramo.lower() if ramo else None
    }

In [11]:
resultado = vagas["description"].apply(processar_descricao).apply(pd.Series)
vagas["descricao_clean"] = resultado["descricao_limpa"]
vagas["ramo_atividade"] = resultado["ramo_atividade"] 

In [12]:
resultado

,descricao_limpa,ramo_atividade
1,"pedreiro.,construções, reformas e reparos. ens...",construcao
2,"desenvolvedor.,a vaga ecessita experiência com...",informatica
3,"estoquista.,vaga aberta para estoquista n. sho...",logistica
4,"vendedor.,atendimento ao público, vendas e pós...",comercio
6,"assistente de logística.,apoio aos motoristas ...",logistica
...,...,...
566059,"consultor de vendas., proatividade. boa comuni...",comercio
566060,"estilista.,estamos contratando estilista para ...",marketing
566061,"recepcionista.,como recepcionista você deverá ...",administrativo
566062,"promotor de vendas.,oportunidade imperdível: p...",comercio


In [13]:
vagas

,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade
1,BNE,4047558,A68BA07D-2515-494E-9B08-C2D509355E92,pedreiro,pedreiro,Descrição Geral\nEmpresa localizada na cidade ...,"pedreiro.,construções, reformas e reparos. ens...",construcao
2,BNE,4196822,4A13075A-B84A-4D8D-9ED4-4109D986634D,desenvolvedor,desenvolvedor,Descrição Geral\nEmpresa localizada na cidade ...,"desenvolvedor.,a vaga ecessita experiência com...",informatica
3,BNE,4152591,3ADD8F96-274C-4AC2-869E-A36CD835F553,estoquista,estoquista,Descrição Geral\nEmpresa localizada na cidade ...,"estoquista.,vaga aberta para estoquista n. sho...",logistica
4,BNE,4225298,AF23DA13-6E63-4D01-8A77-D1950DBCBEC1,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor.,atendimento ao público, vendas e pós...",comercio
6,BNE,4307413,C5F8FA3F-F559-4B36-8661-A9EDDB5E8D68,assistente de logística,assistente logistica,Descrição Geral\nEmpresa localizada na cidade ...,"assistente de logística.,apoio aos motoristas ...",logistica
...,...,...,...,...,...,...,...,...
566059,BNE,3942948,FFA43E9C-3D4A-4D2A-8334-C2FFAF92FF73,consultor de vendas,consultor vendas,Descrição Geral\nEmpresa localizada na cidade ...,"consultor de vendas., proatividade. boa comuni...",comercio
566060,BNE,4383863,A10EDA8C-BF4A-49DD-87C4-1950BF95A6F4,estilista,estilista,Descrição Geral\nEmpresa localizada na cidade ...,"estilista.,estamos contratando estilista para ...",marketing
566061,BNE,4082387,0935370C-414D-4CC4-BA0F-6AA8088573D9,recepcionista,recepcionista,Descrição Geral\nEmpresa localizada na cidade ...,"recepcionista.,como recepcionista você deverá ...",administrativo
566062,BNE,4120573,D710A67D-DCC6-443F-8841-652E8ACBC342,promotor de vendas,promotor vendas,Descrição Geral\nEmpresa localizada na cidade ...,"promotor de vendas.,oportunidade imperdível: p...",comercio


### HABILIDADES PCN

In [14]:
df_habs = pd.read_csv(diretorio+"habilidades_pcn.csv", sep=";")

In [15]:
df_habs

,Habilidade
0,Comunicar-se de maneira assertiva.
1,Utilizar termos técnicos nas rotinas de trabalho.
2,Pesquisar informações e dados relacionados à á...
3,Operar equipamentos fotográficos.
4,"Organizar equipamentos, acessórios e material ..."
...,...
15609,"Diferenciar mercadorias, produtos e serviços."
15610,Proatividade no atendimento e na resolução de ...
15611,Sigilo no tratamento de dados e informações.
15612,Zelo pela segurança do processo de compra.


In [16]:
lista_habilidades = df_habs["Habilidade"].dropna().tolist()
lista_habilidades = [h.strip().lower() for h in lista_habilidades if h.strip() != '']

In [17]:
print(lista_habilidades[:10])

['comunicar-se de maneira assertiva.', 'utilizar termos técnicos nas rotinas de trabalho.', 'pesquisar informações e dados relacionados à área de fotografia.', 'operar equipamentos fotográficos.', 'organizar equipamentos, acessórios e material produzido.', 'identificar cenas fotográficas.', 'harmonizar cores, texturas, formas e iluminação do ambiente.', 'efetuar as operações matemáticas básicas.', 'colaboração no desenvolvimento do trabalho em equipe.', 'cordialidade e empatia no trato com as pessoas.']


In [98]:
client = AzureOpenAI(
    # This is the default and can be omitted
    api_key="d6fd06ab74ac483cb8048f6bcd10ee81",
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2024-08-01-preview",
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint="https://openai-prospeccao.openai.azure.com/",
)

In [19]:
import openai
import pandas as pd
from itertools import chain


# Criar uma função para gerar sinônimos e variações para cada habilidade usando GPT
def gerar_sinonimos(habilidades):
    sinonimos_dict = {}
    
    for habilidade in habilidades[:50]:  # Limitando a 50 habilidades para processamento inicial
        prompt = f"""Liste 5 sinônimos ou variações para a habilidade '{habilidade}', sem introduções ou explicações, apenas a lista:
1. 
2. 
3. 
4. 
5. """

        resposta = client.chat.completions.create(
            model="gpt-4o-OpenAI-Prospeccao",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150
        )

        # Processamento da resposta
        sinonimos = resposta.choices[0].message.content.split("\n")

        # Remover números e espaços extras dos sinônimos
        sinonimos = [s.strip().split(". ", 1)[-1] for s in sinonimos if s.strip()]

        # Garantir que há exatamente 5 sinônimos (preenchendo com strings vazias se necessário)
        while len(sinonimos) < 5:
            sinonimos.append("")

        sinonimos_dict[habilidade] = sinonimos[:5]  # Mantendo apenas os 5 primeiros

    return sinonimos_dict

# Gerar sinônimos e variações
sinonimos_habilidades = gerar_sinonimos(lista_habilidades)

# Transformar em DataFrame para visualização
sinonimos_df = pd.DataFrame.from_dict(sinonimos_habilidades, orient="index", columns=["Sinonimo1", "Sinonimo2", "Sinonimo3", "Sinonimo4", "Sinonimo5"])
sinonimos_df.reset_index(inplace=True)
sinonimos_df.rename(columns={"index": "Habilidade"}, inplace=True)

# Salvar os sinônimos gerados em um novo CSV
sinonimos_df.to_csv("habilidades_expandidas.csv", index=False)

# Exibir amostra dos resultados
print(sinonimos_df.head())

                                          Habilidade  \
0                 comunicar-se de maneira assertiva.   
1  utilizar termos técnicos nas rotinas de trabalho.   
2  pesquisar informações e dados relacionados à á...   
3                  operar equipamentos fotográficos.   
4  organizar equipamentos, acessórios e material ...   

                                           Sinonimo1  \
0              Expressar-se com clareza e confiança.   
1  Empregar jargões profissionais nas atividades ...   
2   Investigar informações e dados sobre fotografia.   
3                                   Manusear câmeras   
4  Gerenciar os equipamentos, acessórios e produt...   

                                           Sinonimo2  \
0            Articular-se de forma direta e honesta.   
1  Aplicar vocabulário especializado nas operaçõe...   
2          Coletar dados e informações fotográficas.   
3                 Utilizar dispositivos fotográficos   
4  Classificar os equipamentos, acessórios e m

In [20]:
# Criar uma lista única de habilidades e seus sinônimos
habilidades_expandida = set()

for _, row in sinonimos_df.iterrows():
    habilidade_principal = row["Habilidade"].strip().lower()
    sinonimos = [str(row[f"Sinonimo{i}"]).strip().lower() for i in range(1, 6) if pd.notna(row[f"Sinonimo{i}"])]

    # Adicionar a habilidade principal e os sinônimos à lista
    habilidades_expandida.add(habilidade_principal)
    habilidades_expandida.update(sinonimos)

# Converter para lista ordenada para garantir consistência
habilidades_expandida = sorted(habilidades_expandida)

# Exibir algumas habilidades para validação
print(habilidades_expandida[:10])

['abertura à diversidade humana.', 'abordagem proativa no atendimento.', 'aceitação das singularidades.', 'acessibilidade e humanidade', 'adaptabilidade no ambiente profissional', 'adequar equipamentos tecnológicos', 'adesão aos regulamentos de patentes e marcas.', 'administração de workflows.', 'agilidade na gestão de questões', 'ajustar sistemas eletrônicos']


In [58]:
df_habilidades_expandidas = pd.DataFrame(habilidades_expandida)

In [60]:
df_habilidades_expandidas.to_csv(diretorio + 'habilidades_expandidas.csv', sep=";", encoding = 'utf-8-sig')

## GERANDO BASE DE TREINO COM CHATGPT

In [61]:
from sklearn.model_selection import train_test_split

# Remove NaNs e ramos com apenas uma ocorrência
ramos_validos = vagas["ramo_atividade"].value_counts()
ramos_validos = ramos_validos[ramos_validos >= 2].index
vagas_filtradas = vagas[vagas["ramo_atividade"].isin(ramos_validos)].dropna(subset=["ramo_atividade"])

# Amostragem estratificada com 3000 vagas
base_retreino, _ = train_test_split(
    vagas_filtradas,
    train_size=3000,
    stratify=vagas_filtradas["ramo_atividade"],
    random_state=42
)

base_retreino = base_retreino.copy()

In [62]:
prompt_competencia_gpt = (
    "Você é um especialista em Processamento de Linguagem Natural (NLP) aplicado ao mercado de trabalho. "
    
    "Sua tarefa é analisar descrições de vagas de emprego e identificar, de forma estruturada, cinco categorias de informações relacionadas ao perfil profissional desejado:\n\n"

    "1. **Habilidades**: capacidades técnicas ou práticas que o candidato precisa demonstrar, como 'atendimento ao cliente', 'operar máquinas', 'vendas', 'comunicação assertiva';\n"
    "2. **Conhecimentos**: saberes técnicos, normativos ou teóricos necessários à função, como 'Excel', 'normas da Anvisa', 'legislação trabalhista'. **Não inclua níveis de escolaridade aqui**;\n"
    "3. **Atitudes e Valores**: comportamentos e disposições esperadas do profissional, como 'proatividade', 'trabalho em equipe', 'responsabilidade', 'ética', 'comprometimento';\n"
    "4. **Escolaridade**: grau de instrução exigido, como 'ensino fundamental completo', 'ensino médio completo', 'ensino superior', 'pós-graduação', 'mestrado', 'doutorado';\n"
    "5. **Certificações**: certificados técnicos ou profissionais específicos, como 'CPA10', 'CREA', 'CRT', 'SAP', 'ISO'.\n\n"

    "Ignore informações que **não** dizem respeito diretamente ao perfil do candidato, como:\n"
    "- Nome da empresa;\n"
    "- Localização da vaga;\n"
    "- Jornada de trabalho;\n"
    "- Tipo de contrato;\n"
    "- Benefícios;\n"
    "- Requisitos vagos como “experiência prévia” sem detalhamento da atividade.\n\n"

    "Você pode usar a seguinte lista de referência com habilidades, conhecimentos e atitudes, derivada de currículos oficiais de formação profissional:\n"
    f"{habilidades_expandida}\n\n"

    "Alem disso, você deve fazer um processo similar com o que está na base:\n" f"{base_treino}\n\n"

    "Use essa lista como apoio para reconhecer termos similares mesmo com variações na linguagem.\n\n"

    "Retorne o resultado estruturado neste formato (sem explicações adicionais):\n"
    "Habilidades: [lista]\n"
    "Conhecimentos: [lista]\n"
    "Atitudes e Valores: [lista]\n"
    "Escolaridade: [lista]\n"
    "Certificações: [lista]\n"
)

# Função que faz a chamada para o GPT
def analisar_vaga_gpt(descricao):
    if not isinstance(descricao, str) or descricao.strip() == "":
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade":[], "Certificacoes": []}

    try:
        response = client.chat.completions.create(
            model="gpt-4o-OpenAI-Prospeccao",  # nome exato do seu deployment no Azure
            messages=[
                {"role": "system", "content": prompt_competencia_gpt},
                {"role": "user", "content": descricao}
            ],
            temperature=0,
            max_tokens=800
        )
        content = response.choices[0].message.content

        # Parsing simples (pode melhorar depois com regex se quiser)
        resultado = {
            "Habilidades": [],
            "Conhecimentos": [],
            "Atitudes_Valores": [],
            "Escolaridade":[], 
            "Certificacoes": []
        }
        for linha in content.split("\n"):
            if linha.lower().startswith("habilidades"):
                resultado["Habilidades"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif linha.lower().startswith("conhecimentos"):
                resultado["Conhecimentos"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif "atitudes" in linha.lower():
                resultado["Atitudes_Valores"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif "escolaridade" in linha.lower():
                resultado["Escolaridade"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif "certificacoes" in linha.lower():
                resultado["Certificacoes"] = linha.split(":", 1)[-1].strip(" []").split(", ")
        return resultado
    except Exception as e:
        print(f"Erro: {e}")
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade": [], "Certificacoes": []}

In [65]:
vagas_amostra = vagas.sample(200).copy()

In [66]:
from tqdm import tqdm
tqdm.pandas()
vagas_amostra["competencias_extraidas"] = vagas_amostra["description"].progress_apply(analisar_vaga_gpt)

100%|██████████| 200/200 [25:00<00:00,  7.50s/it]


In [67]:
check = vagas_amostra[['description', 'competencias_extraidas']]

In [68]:
check.to_csv(diretorio + 'check.csv', sep=";", encoding='utf-8-sig')

In [69]:
check = pd.read_csv(diretorio + 'check.csv', sep=";")

In [70]:
check

,Unnamed: 0,description,competencias_extraidas
0,374626,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['elaborar projetos', 'desenvo..."
1,14366,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['alinhamento de pneus', 'subs..."
2,557333,Descrição Geral\nEmpresa localizada na cidade ...,{'Habilidades': ['fiscalizar entrada e saída d...
3,504463,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '..."
4,544138,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['vendas', 'telemarketing'], '..."
...,...,...,...
195,292208,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['montagem de máquinas', 'mont..."
196,545420,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['manutenção preventiva', 'man..."
197,254989,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '..."
198,128300,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '..."


In [ ]:
# Filtrar linhas com ajustes preenchidos
exemplos_ajustados = check[check["ajustes"].notna()].copy()

# Selecionar amostra para few-shot prompting
amostra_exemplos = exemplos_ajustados.sample(n=min(30, len(exemplos_ajustados)), random_state=42)

amostra_exemplos


,indice,description,competencias_extraidas,ajustes
2,449017,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['vendas', 'atendimento ao cli...","{'Habilidades': ['vendas', 'prospecção de clie..."
30,3849,Descrição Geral\nEmpresa localizada na cidade ...,{'Habilidades': ['manusear produtos de limpeza...,"{'Habilidades': ['limpeza em geral', 'manusear..."
124,144427,Descrição Geral\nEmpresa localizada na cidade ...,{'Habilidades': ['analisar o desenvolvimento d...,{'Habilidades': ['analisar o desenvolvimento d...
64,127254,Descrição Geral\nEmpresa localizada na cidade ...,{'Habilidades': ['liderança e administração da...,{'Habilidades': ['apoiar o engenheiro gestor a...
146,323031,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['organizar veículos', 'zelar ...","{'Habilidades': ['manobrar veículos', 'organiz..."
167,342703,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [""'atendimento ao cliente'"", ""...","{'Habilidades': [""'atendimento ao cliente'"", ""..."
187,136612,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '...",{'Habilidades': ['serviços rotineiros operacio...
94,410391,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': ['prospecção do mercado', 'ven...","{'Habilidades': ['prospecção do mercado', 'ven..."
148,368612,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '...",{'Habilidades': ['limpeza e higienização das d...
63,192602,Descrição Geral\nEmpresa localizada na cidade ...,"{'Habilidades': [''], 'Conhecimentos': [''], '...",{'Habilidades': ['recepção voltada para área d...


### EXPANSÃO DA BASE PARA 3 MIL VAGAS

In [174]:
vagas_amostra2 = vagas.sample(3000)

In [153]:
# Gera o prompt principal com os exemplos ajustados
prompt_base = (
    "Você é um especialista em Processamento de Linguagem Natural (NLP) aplicado ao mercado de trabalho. "
    "Sua tarefa é analisar descrições de vagas de emprego e identificar, de forma estruturada, cinco categorias de informações relacionadas ao perfil profissional desejado:\n\n"
    "1. **Habilidades**: capacidades técnicas ou práticas que o candidato precisa demonstrar, como 'atendimento ao cliente', 'operar máquinas', 'vendas', 'comunicação assertiva';\n"
    "2. **Conhecimentos**: saberes técnicos, normativos ou teóricos necessários à função, como 'Excel', 'normas da Anvisa', 'legislação trabalhista'. **Não inclua níveis de escolaridade aqui**;\n"
    "3. **Atitudes e Valores**: comportamentos e disposições esperadas do profissional, como 'proatividade', 'trabalho em equipe', 'responsabilidade', 'ética', 'comprometimento';\n"
    "4. **Escolaridade**: grau de instrução exigido, como 'ensino fundamental completo', 'ensino médio completo', 'ensino superior', 'pós-graduação', 'mestrado', 'doutorado';\n"
    "5. **Certificações**: certificados técnicos ou profissionais específicos, como 'CPA10', 'CREA', 'CRT', 'SAP', 'ISO'.\n\n"
    "Ignore informações que **não** dizem respeito diretamente ao perfil do candidato, como:\n"
    "- Nome da empresa;\n"
    "- Localização da vaga;\n"
    "- Jornada de trabalho;\n"
    "- Tipo de contrato;\n"
    "- Benefícios;\n"
    "- Requisitos vagos como “experiência prévia” sem detalhamento da atividade.\n\n"
    "Você pode usar a seguinte lista de referência com habilidades, conhecimentos e atitudes derivada de currículos oficiais:\n"
    f"{habilidades_expandida}\n\n"
    "Use essa lista como apoio para reconhecer termos similares mesmo com variações na linguagem.\n\n"
    "Abaixo estão alguns exemplos de como a tarefa deve ser feita com base em descrições reais e seus respectivos resultados ajustados manualmente:\n"
)

# Adiciona exemplos com ajustes
exemplos_ajustados = ""
for _, row in amostra_exemplos.iterrows():
    exemplos_ajustados += f"\nDescrição:\n{row['description']}\n"
    exemplos_ajustados += f"Resultado:\n{row['ajustes'].strip()}\n"

# Prompt final com exemplos + instrução
prompt_com_exemplos = prompt_base + exemplos_ajustados + "\nAgora, analise a próxima vaga:\n"

In [170]:
import re

def analisar_vaga_gpt_com_exemplos(descricao):
    if not isinstance(descricao, str) or descricao.strip() == "":
        return {
            "Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [],
            "Escolaridade": [], "Certificacoes": []
        }

    try:
        response = client.chat.completions.create(
            model="gpt-4o-OpenAI-Prospeccao",
            messages=[
                {"role": "system", "content": prompt_com_exemplos},
                {"role": "user", "content": descricao}
            ],
            temperature=0,
            max_tokens=1000
        )
        content = response.choices[0].message.content

        # Inicializa o dicionário final
        resultado = {
            "Habilidades": [],
            "Conhecimentos": [],
            "Atitudes_Valores": [],
            "Escolaridade": [],
            "Certificacoes": []
        }

        # Pré-processa todo o conteúdo para remover colchetes e aspas
        content = content.replace("[", "").replace("]", "").replace("'", "").replace('"', '').strip()

        # Separar o conteúdo inteiro por linhas
        linhas = content.split("\n")
        secao_atual = None

        for linha in linhas:
            linha = linha.strip()
            if not linha:
                continue

            if ":" in linha:
                chave, valor = linha.split(":", 1)
                chave = chave.strip().lower()
                valor = valor.strip()

                if "habilidade" in chave:
                    secao_atual = "Habilidades"
                elif "conhecimento" in chave:
                    secao_atual = "Conhecimentos"
                elif "atitude" in chave:
                    secao_atual = "Atitudes_Valores"
                elif "escolaridade" in chave:
                    secao_atual = "Escolaridade"
                elif "certificacao" in chave:
                    secao_atual = "Certificacoes"
                else:
                    secao_atual = None

                if secao_atual and valor:
                    resultado[secao_atual] = [item.strip() for item in valor.split(",") if item.strip()]
            elif secao_atual:
                # Se não tem ":", mas é continuação de uma seção válida
                resultado[secao_atual].extend(
                    [item.strip() for item in linha.split(",") if item.strip()]
                )

        return resultado

    except Exception as e:
        print(f"Erro: {e}")
        return {
            "Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [],
            "Escolaridade": [], "Certificacoes": []
        }

In [175]:
vagas_amostra2["competencias_extraidas"] = vagas_amostra2["description"].progress_apply(analisar_vaga_gpt_com_exemplos)

  6%|▌         | 186/3000 [1:03:05<18:17:38, 23.40s/it]

Erro: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


 17%|█▋        | 507/3000 [2:53:08<20:51:12, 30.11s/it]

Erro: Connection error.


 62%|██████▏   | 1871/3000 [10:48:22<6:06:27, 19.48s/it] 

Erro: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


 84%|████████▍ | 2513/3000 [14:27:40<2:40:17, 19.75s/it]

Erro: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


100%|██████████| 3000/3000 [17:14:30<00:00, 20.69s/it]  


In [242]:
vagas_amostra2 = pd.read_csv(diretorio + 'base_treino_gpt.csv', sep=";")
vagas_amostra2

,Unnamed: 0.1,Unnamed: 0,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,competencias_extraidas
0,0,89329,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"{'Habilidades': ['manuseio de chapa', 'manusei..."
1,1,363048,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,"{'Habilidades': ['desenho arquitetônico', 'Con..."
2,2,488515,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"{'Habilidades': ['eventos rotina dp em geral',..."
3,3,546955,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"{'Habilidades': ['manutenção de interfones', '..."
4,4,396472,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,"{'Habilidades': ['atendimento ao público', 'Co..."
...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,387671,BNE,4219906,B3BDC1E5-B1FB-4DC3-AE4B-6C70EA46D550,montador de móveis,montador moveis,Descrição Geral\nEmpresa localizada na cidade ...,"montador de móveis.,vaga para montador de móve...",moveis,"{'Habilidades': ['montagem de móveis', 'Conhec..."
2996,2996,295233,BNE,4391549,8F476A07-0915-41FF-9243-EE2EA7A5827A,operador de caixa,operador caixa,Descrição Geral\nEmpresa localizada na cidade ...,"operador de caixa.,empresa: cinco estrelas cas...",financeiro,"{'Habilidades': ['Conhecimentos:', 'Atitudes_V..."
2997,2997,106118,BNE,4026603,ECAC7277-9D5E-4142-A022-CB237E190317,auxiliar comercial,auxiliar comercial,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar comercial.,suporte ao time comercial,...",comercio,"{'Habilidades': ['suporte ao time comercial', ..."
2998,2998,457735,BNE,4029892,F280B1CE-C87A-4FFE-914A-DE929D9A3E2A,Auxiliar de Contabilidade,auxiliar contabilidade,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de contabilidade.,experiência em trib...",contabilidade,{'Habilidades': ['importação de folha de pagam...


In [243]:
teste = vagas_amostra2

In [244]:
teste["competencias_extraidas"] = teste["competencias_extraidas"].astype(str).str.extract(r"(.*?})")[0]
teste

,Unnamed: 0.1,Unnamed: 0,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,competencias_extraidas
0,0,89329,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"{'Habilidades': ['manuseio de chapa', 'manusei..."
1,1,363048,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,"{'Habilidades': ['desenho arquitetônico', 'Con..."
2,2,488515,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"{'Habilidades': ['eventos rotina dp em geral',..."
3,3,546955,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"{'Habilidades': ['manutenção de interfones', '..."
4,4,396472,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,"{'Habilidades': ['atendimento ao público', 'Co..."
...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,387671,BNE,4219906,B3BDC1E5-B1FB-4DC3-AE4B-6C70EA46D550,montador de móveis,montador moveis,Descrição Geral\nEmpresa localizada na cidade ...,"montador de móveis.,vaga para montador de móve...",moveis,"{'Habilidades': ['montagem de móveis', 'Conhec..."
2996,2996,295233,BNE,4391549,8F476A07-0915-41FF-9243-EE2EA7A5827A,operador de caixa,operador caixa,Descrição Geral\nEmpresa localizada na cidade ...,"operador de caixa.,empresa: cinco estrelas cas...",financeiro,"{'Habilidades': ['Conhecimentos:', 'Atitudes_V..."
2997,2997,106118,BNE,4026603,ECAC7277-9D5E-4142-A022-CB237E190317,auxiliar comercial,auxiliar comercial,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar comercial.,suporte ao time comercial,...",comercio,"{'Habilidades': ['suporte ao time comercial', ..."
2998,2998,457735,BNE,4029892,F280B1CE-C87A-4FFE-914A-DE929D9A3E2A,Auxiliar de Contabilidade,auxiliar contabilidade,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de contabilidade.,experiência em trib...",contabilidade,{'Habilidades': ['importação de folha de pagam...


In [245]:
teste["competencias_extraidas"] = teste["competencias_extraidas"].str.replace("[", "", regex=False)
teste

,Unnamed: 0.1,Unnamed: 0,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,competencias_extraidas
0,0,89329,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"{'Habilidades': 'manuseio de chapa', 'manuseio..."
1,1,363048,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,"{'Habilidades': 'desenho arquitetônico', 'Conh..."
2,2,488515,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"{'Habilidades': 'eventos rotina dp em geral', ..."
3,3,546955,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"{'Habilidades': 'manutenção de interfones', 'i..."
4,4,396472,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,"{'Habilidades': 'atendimento ao público', 'Con..."
...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,387671,BNE,4219906,B3BDC1E5-B1FB-4DC3-AE4B-6C70EA46D550,montador de móveis,montador moveis,Descrição Geral\nEmpresa localizada na cidade ...,"montador de móveis.,vaga para montador de móve...",moveis,"{'Habilidades': 'montagem de móveis', 'Conheci..."
2996,2996,295233,BNE,4391549,8F476A07-0915-41FF-9243-EE2EA7A5827A,operador de caixa,operador caixa,Descrição Geral\nEmpresa localizada na cidade ...,"operador de caixa.,empresa: cinco estrelas cas...",financeiro,"{'Habilidades': 'Conhecimentos:', 'Atitudes_Va..."
2997,2997,106118,BNE,4026603,ECAC7277-9D5E-4142-A022-CB237E190317,auxiliar comercial,auxiliar comercial,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar comercial.,suporte ao time comercial,...",comercio,"{'Habilidades': 'suporte ao time comercial', '..."
2998,2998,457735,BNE,4029892,F280B1CE-C87A-4FFE-914A-DE929D9A3E2A,Auxiliar de Contabilidade,auxiliar contabilidade,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de contabilidade.,experiência em trib...",contabilidade,{'Habilidades': 'importação de folha de pagame...


In [249]:
# Função para extrair blocos de texto com base nos marcadores em uma string única
def extrair_blocos_de_string(texto):
    try:
        if not isinstance(texto, str):
            return [], [], [], [], []

        # Limpeza inicial
        texto = texto.replace("\n", " ").replace("{", "").replace("}", "").replace("'", "")

        # Divisões usando os marcadores
        partes = re.split(r"(?i)\b(Conhecimentos:|Atitudes_Valores:|Escolaridade:|Certificacoes:)", texto)

        # Inicializar
        habilidades = []
        conhecimentos = []
        atitudes = []
        escolaridade = []
        certificacoes = []

        # Separar blocos
        if partes:
            habilidades_texto = partes[0].replace("Habilidades:", "").strip().strip(",")
            habilidades = [h.strip() for h in habilidades_texto.split(",") if h.strip()]

            for i in range(1, len(partes), 2):
                categoria = partes[i].strip(": ")
                conteudo = partes[i+1] if i+1 < len(partes) else ""
                itens = [c.strip() for c in conteudo.split(",") if c.strip()]

                if categoria.lower() == "conhecimentos":
                    conhecimentos = itens
                elif categoria.lower() == "atitudes_valores":
                    atitudes = itens
                elif categoria.lower() == "escolaridade":
                    escolaridade = itens
                elif categoria.lower() == "certificacoes":
                    certificacoes = itens

        return habilidades, conhecimentos, atitudes, escolaridade, certificacoes

    except Exception:
        return [], [], [], [], []

# Aplicar a função
teste[["Habilidades", "Conhecimentos", "Atitudes_Valores", "Escolaridade", "Certificacoes"]] = teste["competencias_extraidas"].apply(
    lambda x: pd.Series(extrair_blocos_de_string(x))
)

teste

,Unnamed: 0.1,Unnamed: 0,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,competencias_extraidas,Habilidades,Conhecimentos,Atitudes_Valores,Escolaridade,Certificacoes
0,0,89329,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"{'Habilidades': 'manuseio de chapa', 'manuseio...","[manuseio de chapa, manuseio de grelha]",[],[],[],[]
1,1,363048,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,"{'Habilidades': 'desenho arquitetônico', 'Conh...",[desenho arquitetônico],[AutoCAD],[],"[estudante de engenharia civil, estudante de a...",[]
2,2,488515,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"{'Habilidades': 'eventos rotina dp em geral', ...","[eventos rotina dp em geral, processos de admi...","[eSocial, DCTFWeb]",[],[],[]
3,3,546955,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"{'Habilidades': 'manutenção de interfones', 'i...","[manutenção de interfones, instalação de inter...",[interfones condominiais],[],[],[]
4,4,396472,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,"{'Habilidades': 'atendimento ao público', 'Con...",[atendimento ao público],[],"[boa comunicação, bom relacionamento com a equ...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,387671,BNE,4219906,B3BDC1E5-B1FB-4DC3-AE4B-6C70EA46D550,montador de móveis,montador moveis,Descrição Geral\nEmpresa localizada na cidade ...,"montador de móveis.,vaga para montador de móve...",moveis,"{'Habilidades': 'montagem de móveis', 'Conheci...",[montagem de móveis],[],[],[],[]
2996,2996,295233,BNE,4391549,8F476A07-0915-41FF-9243-EE2EA7A5827A,operador de caixa,operador caixa,Descrição Geral\nEmpresa localizada na cidade ...,"operador de caixa.,empresa: cinco estrelas cas...",financeiro,"{'Habilidades': 'Conhecimentos:', 'Atitudes_Va...",[],[],[],[],[]
2997,2997,106118,BNE,4026603,ECAC7277-9D5E-4142-A022-CB237E190317,auxiliar comercial,auxiliar comercial,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar comercial.,suporte ao time comercial,...",comercio,"{'Habilidades': 'suporte ao time comercial', '...","[suporte ao time comercial, realizar contatos ...",[],[],[],[]
2998,2998,457735,BNE,4029892,F280B1CE-C87A-4FFE-914A-DE929D9A3E2A,Auxiliar de Contabilidade,auxiliar contabilidade,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de contabilidade.,experiência em trib...",contabilidade,{'Habilidades': 'importação de folha de pagame...,"[importação de folha de pagamento, conciliação...","[tributos fiscais, DIRF, cálculo de IR]",[],[],[]


In [250]:
teste.to_csv(diretorio + 'base_treino_gpt2.csv', sep=";", encoding= 'utf-8-sig')

## após alguns pequenos ajustes manuais, carregar a base novamente

In [71]:
base_treino = pd.read_csv(diretorio + 'base_treino_gpt2.csv', sep=";", encoding='utf-8-sig')

In [74]:
base_retreino

,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade
200657,BNE,4207444,23C853EC-9254-4621-9F04-AECD0E3662C2,Gerente Customer Service,gerente customer servico,Descrição Geral\nEmpresa localizada na cidade ...,"gerente customer service.,somos uma operadora ...",comercio
474712,BNE,4209889,13AF5674-2DFA-42DE-BA9F-73F659B3E15F,Doméstica,domestica,Descrição Geral\nEmpresa localizada na cidade ...,"doméstica.,teste teste teste teste teste teste...",limpeza
480029,BNE,4312823,41062C74-E96D-40E0-A73E-0B12C5815D9E,comprador,comprador,Descrição Geral\nEmpresa localizada na cidade ...,"comprador.,desejável vivência o segmento. ativ...",administrativo
192040,BNE,3956220,9BC334F9-CD24-4E13-B2B0-63BA885290AC,auxiliar de serviços gerais,auxiliar servicos gerais,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de serviços gerais.,responsável pela ...",limpeza
354166,BNE,4311442,E7F70304-A762-46BC-9F9A-139D22232A1E,mecânico de manutenção,mecanico manutencao,Descrição Geral\nEmpresa localizada na cidade ...,"mecânico de manutenção.,vaga para atuar em man...",manutencao
...,...,...,...,...,...,...,...,...
204796,BNE,4244618,E7AD5B8A-F3CC-4A01-B35A-5DBD2ECEEDF2,Estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário.,estágio atendimentor r atividades:...",administrativo
6180,BNE,4241885,FA7425AA-C60C-4BC0-B3EF-C934C393B69D,auxiliar de almoxarifado,auxiliar almoxarifado,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de almoxarifado.,vaga para auxiliar d...",logistica
56915,BNE,3964298,0F099222-F1D8-4C06-9CD3-B0B9B43460B7,vigia,vigia,Descrição Geral\nEmpresa localizada na cidade ...,"vigia.,vaga para atuar como vigia, escala 12x3...",seguranca
303536,BNE,4026387,FD10E4EB-8815-47C0-BC14-F20535150B06,açougueiro,acougueiro,Descrição Geral\nEmpresa localizada na cidade ...,"açougueiro.,experiência em atendimentos e cort...",alimentos


In [103]:
# 1. Carrega as bases
vagas = pd.read_csv(diretorio + 'vagas.csv', sep=";", encoding="utf-8-sig")

# 3. Amostragem estratificada com no mínimo 4 por grupo
min_grupo = 4
grupos_validos = vagas['ramo_atividade'].value_counts()
grupos_validos = grupos_validos[grupos_validos >= min_grupo].index

vagas_filtradas = vagas[vagas['ramo_atividade'].isin(grupos_validos)]

vagas_sample = (
    vagas_filtradas
    .groupby('ramo_atividade', group_keys=False)
    .apply(lambda x: x.sample(frac=100/len(vagas_filtradas), random_state=42))
    .reset_index(drop=True)
)

In [105]:
vagas_sample.to_csv(diretorio + 'vagas_sample100.csv', sep=";", encoding = 'utf-8')

In [75]:
from tqdm import tqdm
import pandas as pd

# Inicializa colunas
base_retreino["Habilidades"] = ""
base_retreino["Conhecimentos"] = ""
base_retreino["Atitudes_Valores"] = ""
base_retreino["Escolaridade"] = ""
base_retreino["Certificacoes"] = ""

# Aplicação linha a linha com progresso
for idx, row in tqdm(base_retreino.iterrows(), total=len(base_retreino)):
    descricao = row["descricao_clean"]
    resultado = analisar_vaga_gpt(descricao)

    base_retreino.at[idx, "Habilidades"] = str(resultado["Habilidades"])
    base_retreino.at[idx, "Conhecimentos"] = str(resultado["Conhecimentos"])
    base_retreino.at[idx, "Atitudes_Valores"] = str(resultado["Atitudes_Valores"])
    base_retreino.at[idx, "Escolaridade"] = str(resultado["Escolaridade"])
    base_retreino.at[idx, "Certificacoes"] = str(resultado["Certificacoes"])

    if (idx + 1) % 30 == 0:
        print(f">>> Processadas {idx + 1} descrições")

# Exporta no padrão da anterior
base_retreino.to_csv(diretorio +"base_retreino_gpt.csv", sep=";", encoding="utf-8-sig", index=False)
print("✅ Base salva como base_retreino_gpt.csv")

  0%|          | 3/3000 [00:08<1:52:13,  2.25s/it]

>>> Processadas 480030 descrições


  0%|          | 6/3000 [00:14<1:33:53,  1.88s/it]

Erro: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


  0%|          | 14/3000 [01:16<3:46:38,  4.55s/it]

...

>>> Processadas 13890 descrições


 98%|█████████▊| 2954/3000 [6:17:08<07:27,  9.72s/it]

>>> Processadas 346530 descrições


100%|██████████| 3000/3000 [6:22:49<00:00,  7.66s/it]


✅ Base salva como base_retreino_gpt.csv


In [76]:
base_retreino

,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,Habilidades,Conhecimentos,Atitudes_Valores,Escolaridade,Certificacoes
200657,BNE,4207444,23C853EC-9254-4621-9F04-AECD0E3662C2,Gerente Customer Service,gerente customer servico,Descrição Geral\nEmpresa localizada na cidade ...,"gerente customer service.,somos uma operadora ...",comercio,"['medir e gerar indicadores de experiência', '...","['Excel avançado', 'Power BI', 'PowerPoint']","['proatividade', 'organização', 'trabalho em e...",[''],[]
474712,BNE,4209889,13AF5674-2DFA-42DE-BA9F-73F659B3E15F,Doméstica,domestica,Descrição Geral\nEmpresa localizada na cidade ...,"doméstica.,teste teste teste teste teste teste...",limpeza,[''],[''],[''],[''],[]
480029,BNE,4312823,41062C74-E96D-40E0-A73E-0B12C5815D9E,comprador,comprador,Descrição Geral\nEmpresa localizada na cidade ...,"comprador.,desejável vivência o segmento. ativ...",administrativo,"[""'realizar cotações'"", ""'fechamento de compra...","[""'segmento de alimentos'"", ""'produtos de limp...",[''],[''],[]
192040,BNE,3956220,9BC334F9-CD24-4E13-B2B0-63BA885290AC,auxiliar de serviços gerais,auxiliar servicos gerais,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de serviços gerais.,responsável pela ...",limpeza,"[""'manutenção da limpeza'""]",[''],[''],"[""'ensino fundamental incompleto'""]",[]
354166,BNE,4311442,E7F70304-A762-46BC-9F9A-139D22232A1E,mecânico de manutenção,mecanico manutencao,Descrição Geral\nEmpresa localizada na cidade ...,"mecânico de manutenção.,vaga para atuar em man...",manutencao,"[""'manutenção mecânica de equipamentos de solo...","[""'geradores'"", ""'motor diesel'"", ""'hidráulico...",[''],"[""'ensino médio completo'"", ""'técnico completo'""]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204796,BNE,4244618,E7AD5B8A-F3CC-4A01-B35A-5DBD2ECEEDF2,Estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário.,estágio atendimentor r atividades:...",administrativo,"['auxiliar o atendimento ao cliente', 'organiz...",[''],"['disponibilidade manhã', 'tarde']",[''],[]
6180,BNE,4241885,FA7425AA-C60C-4BC0-B3EF-C934C393B69D,auxiliar de almoxarifado,auxiliar almoxarifado,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de almoxarifado.,vaga para auxiliar d...",logistica,"[""'recebimento de mercadorias'"", ""'entregas de...","[""'sistemas de emissão de OS'""]",[''],"[""'ensino médio completo'""]",[]
56915,BNE,3964298,0F099222-F1D8-4C06-9CD3-B0B9B43460B7,vigia,vigia,Descrição Geral\nEmpresa localizada na cidade ...,"vigia.,vaga para atuar como vigia, escala 12x3...",seguranca,[''],[''],[''],[''],[]
303536,BNE,4026387,FD10E4EB-8815-47C0-BC14-F20535150B06,açougueiro,acougueiro,Descrição Geral\nEmpresa localizada na cidade ...,"açougueiro.,experiência em atendimentos e cort...",alimentos,"[""'atendimentos'"", ""'cortes de carne'""]",[''],[''],[''],[]


In [77]:
base_retreino.to_csv(diretorio +"base_retreino_gpt.csv", sep=";", encoding="utf-8-sig", index=False)

In [22]:
base_treino

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,competencias_extraidas,Habilidades,Conhecimentos,Atitudes_Valores,Escolaridade,Certificacoes
0,0,0,89329,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"{'Habilidades': 'manuseio de chapa', 'manuseio...","['manuseio de chapa', 'manuseio de grelha']",[],[],[],[]
1,1,1,363048,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,"{'Habilidades': 'desenho arquitetônico', 'Conh...",['desenho arquitetônico'],['AutoCAD'],[],"['estudante de engenharia civil', 'estudante d...",[]
2,2,2,488515,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"{'Habilidades': 'eventos rotina dp em geral', ...","['eventos rotina dp em geral', 'processos de a...","['eSocial', 'DCTFWeb']",[],[],[]
3,3,3,546955,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"{'Habilidades': 'manutenção de interfones', 'i...","['manutenção de interfones', 'instalação de in...",['interfones condominiais'],[],[],[]
4,4,4,396472,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,"{'Habilidades': 'atendimento ao público', 'Con...",['atendimento ao público'],[],"['boa comunicação', 'bom relacionamento com a ...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,2995,387671,BNE,4219906,B3BDC1E5-B1FB-4DC3-AE4B-6C70EA46D550,montador de móveis,montador moveis,Descrição Geral\nEmpresa localizada na cidade ...,"montador de móveis.,vaga para montador de móve...",moveis,"{'Habilidades': 'montagem de móveis', 'Conheci...",['montagem de móveis'],[],[],[],[]
2996,2996,2996,295233,BNE,4391549,8F476A07-0915-41FF-9243-EE2EA7A5827A,operador de caixa,operador caixa,Descrição Geral\nEmpresa localizada na cidade ...,"operador de caixa.,empresa: cinco estrelas cas...",financeiro,"{'Habilidades': 'Conhecimentos:', 'Atitudes_Va...",[],[],[],[],[]
2997,2997,2997,106118,BNE,4026603,ECAC7277-9D5E-4142-A022-CB237E190317,auxiliar comercial,auxiliar comercial,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar comercial.,suporte ao time comercial,...",comercio,"{'Habilidades': 'suporte ao time comercial', '...","['suporte ao time comercial', 'realizar contat...",[],[],[],[]
2998,2998,2998,457735,BNE,4029892,F280B1CE-C87A-4FFE-914A-DE929D9A3E2A,Auxiliar de Contabilidade,auxiliar contabilidade,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de contabilidade.,experiência em trib...",contabilidade,{'Habilidades': 'importação de folha de pagame...,"['importação de folha de pagamento', 'concilia...","['tributos fiscais', 'DIRF', 'cálculo de IR']",[],[],[]


In [79]:
base_treino = base_treino[
    ['source', 'identifier', 'uuid', 'title', 'title_standardized', 'description', 'descricao_clean',
     'ramo_atividade', 'Habilidades', 'Conhecimentos', 'Atitudes_Valores', 'Escolaridade', 'Certificacoes']
]


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [80]:
base_treino_final = pd.concat([base_treino, base_retreino], ignore_index=True)
base_treino_final

,source,identifier,uuid,title,title_standardized,description,descricao_clean,ramo_atividade,Habilidades,Conhecimentos,Atitudes_Valores,Escolaridade,Certificacoes
0,BNE,4346041,C9F9D669-B185-4078-AD4C-882F014273E7,chapeiro,chapeiro,Descrição Geral\nEmpresa localizada na cidade ...,"chapeiro.,vaga para chapeiro de horário oturno...",metal mecanico,"['manuseio de chapa', 'manuseio de grelha']",[],[],[],[]
1,BNE,4307019,52C0DAAA-12D0-411A-BC9C-E0CFE3BDE34B,estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário., ser estudante de engenharia civil...",administrativo,['desenho arquitetônico'],['AutoCAD'],[],"['estudante de engenharia civil', 'estudante d...",[]
2,BNE,4385222,22899E4A-1DCB-4EC0-AD40-D1FB3104BE42,analista de departamento pessoal,analista departamento pessoal,Descrição Geral\nEmpresa localizada na cidade ...,"analista de departamento pessoal.,atividades: ...",recursos humanos,"['eventos rotina dp em geral', 'processos de a...","['eSocial', 'DCTFWeb']",[],[],[]
3,BNE,3967618,49D0ED16-087C-4E99-A28B-5BE7A29F9226,Suporte Técnico,suporte tecnico,Descrição Geral\nEmpresa localizada na cidade ...,"suporte técnico.,com conhecimento técnico e pr...",informatica,"['manutenção de interfones', 'instalação de in...",['interfones condominiais'],[],[],[]
4,BNE,4047696,9FC80823-A716-40C6-8F92-45F59A49AE2B,vendedor,vendedor,Descrição Geral\nEmpresa localizada na cidade ...,"vendedor., boa comunicação, disponibilidade de...",comercio,['atendimento ao público'],[],"['boa comunicação', 'bom relacionamento com a ...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,BNE,4244618,E7AD5B8A-F3CC-4A01-B35A-5DBD2ECEEDF2,Estagiário,estagiario,Descrição Geral\nEmpresa localizada na cidade ...,"estagiário.,estágio atendimentor r atividades:...",administrativo,"['auxiliar o atendimento ao cliente', 'organiz...",[''],"['disponibilidade manhã', 'tarde']",[''],[]
5996,BNE,4241885,FA7425AA-C60C-4BC0-B3EF-C934C393B69D,auxiliar de almoxarifado,auxiliar almoxarifado,Descrição Geral\nEmpresa localizada na cidade ...,"auxiliar de almoxarifado.,vaga para auxiliar d...",logistica,"[""'recebimento de mercadorias'"", ""'entregas de...","[""'sistemas de emissão de OS'""]",[''],"[""'ensino médio completo'""]",[]
5997,BNE,3964298,0F099222-F1D8-4C06-9CD3-B0B9B43460B7,vigia,vigia,Descrição Geral\nEmpresa localizada na cidade ...,"vigia.,vaga para atuar como vigia, escala 12x3...",seguranca,[''],[''],[''],[''],[]
5998,BNE,4026387,FD10E4EB-8815-47C0-BC14-F20535150B06,açougueiro,acougueiro,Descrição Geral\nEmpresa localizada na cidade ...,"açougueiro.,experiência em atendimentos e cort...",alimentos,"[""'atendimentos'"", ""'cortes de carne'""]",[''],[''],[''],[]


In [108]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------------------- ------- 1.3/1.6 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.2 MB/s eta 0:00:00


In [111]:
base_treino_final_sample = base_treino_final.sample(10).copy()

In [122]:
import pandas as pd
import ast
import re
from rapidfuzz import fuzz, process

# Extrair termos únicos das colunas
def extrair_termos(coluna):
    termos = []
    for lista in base_treino_final[coluna]:
        if pd.isna(lista):
            continue
        try:
            termos += ast.literal_eval(lista)
        except:
            continue
    return list(set([t.strip() for t in termos if isinstance(t, str) and len(t) >= 3]))

termos_habilidades = extrair_termos("Habilidades")
termos_conhecimentos = extrair_termos("Conhecimentos")
termos_atitudes = extrair_termos("Atitudes_Valores")

print("Habilidades:", termos_habilidades[:5])
print("Conhecimentos:", termos_conhecimentos[:5])
print("Atitudes/Valores:", termos_atitudes[:5])

# Função para fuzzy matching
def fuzzy_spans(texto, termos, label, threshold=70):
    spans = []
    for termo in termos:
        matches = process.extract_iter(termo, texto, scorer=fuzz.partial_ratio)
        for match, score, start in matches:
            if score >= threshold:
                end = start + len(match)
                if end - start >= 3:
                    spans.append((start, end, label))
    return spans

# Aplicar aos textos
dados_bio = []
for i, row in tqdm(base_treino_final_sample.iterrows(), total=base_treino_final_sample.shape[0], desc="Processando spans"):
    texto = str(row["description"])
    spans = []
    spans += fuzzy_spans(texto, termos_habilidades, "HABILIDADE")
    spans += fuzzy_spans(texto, termos_conhecimentos, "CONHECIMENTO")
    spans += fuzzy_spans(texto, termos_atitudes, "ATITUDE_VALOR")

    # Remover overlaps
    spans = list(set(spans))
    spans = sorted(spans, key=lambda x: (x[0], -x[1]))
    resultado = []
    ultimo_fim = -1
    for start, end, label in spans:
        if start >= ultimo_fim:
            resultado.append((start, end, label))
            ultimo_fim = end

    if resultado:
        dados_bio.append({"text": texto, "spans": resultado})

Habilidades: ['operação de caixa', 'análise técnica de ordens de serviço', "'liderar negociações'", 'atender clientes', "'executar atividades de panfletagem'"]
Conhecimentos: ["'Go'", 'operação de caixa', 'sistema redesim', 'elétricos e eletrônicos de veículos', "'alterações e transformações societárias'"]
Atitudes/Valores: ["'liberdade'", 'foco em qualidade e excelência', "'facilidade para trabalhar com sistemas'", 'acostumado a trabalhar com metas agressivas]', "'facilidade de comunicação'"]


Processando spans: 100%|██████████| 10/10 [04:53<00:00, 29.36s/it]


In [121]:
texto_exemplo = base_treino_final["description"].iloc[1021]
print("Texto exemplo:", texto_exemplo)

print("Spans habilidade:", fuzzy_spans(texto_exemplo, termos_habilidades, "HABILIDADE"))


Texto exemplo: Descrição Geral\nEmpresa localizada na cidade de Guaçuí/ES do ramo Comércio, contrata Representante de Vendas.,Atribuições\na *tambasa* é uma empresa atacadista, que está a   no mercado comercial e atua em todo o território nacional. está com uma excelente oportunidade para você vendedor atonomo. \r\n ??segmentos:\r\n *construção, produtos do lar e do campo:*\r\n?? agropecuária e veterinária \r\n??fertilizantes, ferragens agrícolas\r\n??caça, pesca e montaria\r\n??calçados, matérias segurança\r\n??material de construção em geral\r\n??pinturas e acabamentos \r\n??eletrodomésticos \r\n?? material escolar, escritório,\r\n?? utilidade domésticas, loucas, vidros, armarinhos, etc.\r\n?? mix com mais de 29 mil itens\r\n?? vantagens:\r\n ??comissão paga semanalmente sobre o faturado \r\n??premiação semanal, mensal, trimestral, semestral, anual\r\n??campanhas de estímulos \r\n??requisitos:\r\ncore ou firma (cnpj)\r\na combinar\r\n?? (informação confidencial)\r\n?? instagram:\r\n@

In [116]:
# Transformar em DataFrame e visualizar
df_resultado = pd.DataFrame(dados_bio)
print(df_resultado.head(3))  # Mostra primeiras 3 vagas

Empty DataFrame
Columns: []
Index: []


In [88]:
base_treino_final.to_csv(diretorio + 'base_treino_final.csv', sep=";", encoding='utf-8-sig')

## AMPLIANDO A BASE

In [ ]:
vagas = pd.read_csv('vagas.csv', sep=";")
vagas_gpt = vagas.sample(6000)
habilidades_expandida = pd.read_csv('habilidades_expandidas.csv', sep=";")

In [ ]:

# === Prompt de contexto ===
prompt_competencia_gpt = (
    "Você é um especialista em Processamento de Linguagem Natural (NLP) aplicado ao mercado de trabalho. "
    "Sua tarefa é analisar descrições de vagas de emprego e identificar, de forma estruturada, cinco categorias de informações relacionadas ao perfil profissional desejado:\n\n"
    "1. **Habilidades**: capacidades técnicas ou práticas que o candidato precisa demonstrar, como 'atendimento ao cliente', 'operar máquinas', 'vendas', 'comunicação assertiva';\n"
    "2. **Conhecimentos**: saberes técnicos, normativos ou teóricos necessários à função, como 'Excel', 'normas da Anvisa', 'legislação trabalhista'. **Não inclua níveis de escolaridade aqui**;\n"
    "3. **Atitudes e Valores**: comportamentos e disposições esperadas do profissional, como 'proatividade', 'trabalho em equipe', 'responsabilidade', 'ética', 'comprometimento';\n\n"
    "Ignore informações que **não** dizem respeito diretamente ao perfil do candidato, como:\n"
    "- Nome da empresa;\n"
    "- Localização da vaga;\n"
    "- Jornada de trabalho;\n"
    "- Tipo de contrato;\n"
    "- Benefícios;\n"
    "- Requisitos vagos como “experiência prévia” sem detalhamento da atividade.\n\n"
    "Retorne o resultado estruturado neste formato (sem explicações adicionais):\n"
    "Habilidades: [lista]\n"
    "Conhecimentos: [lista]\n"
    "Atitudes e Valores: [lista]\n"
)

# === Função GPT ===
def analisar_vaga_gpt(descricao):
    if not isinstance(descricao, str) or descricao.strip() == "":
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade": [], "Certificacoes": []}
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt_competencia_gpt},
                {"role": "user", "content": descricao}
            ],
            temperature=0,
            max_tokens=800
        )
        content = response.choices[0].message.content

        resultado = {
            "Habilidades": [],
            "Conhecimentos": [],
            "Atitudes_Valores": []
        }
        for linha in content.split("\n"):
            if linha.lower().startswith("habilidades"):
                resultado["Habilidades"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif linha.lower().startswith("conhecimentos"):
                resultado["Conhecimentos"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif "atitudes" in linha.lower():
                resultado["Atitudes_Valores"] = linha.split(":", 1)[-1].strip(" []").split(", ")
        return resultado
    except Exception as e:
        print(f"Erro: {e}")
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade": [], "Certificacoes": []}


# === Carregar progresso anterior (se existir) ===
caminho_saida = "resultado_gpt_parcial.csv"
try:
    resultado_parcial = pd.read_csv(caminho_saida, encoding="utf-8-sig")
    processados = set(resultado_parcial["uuid"].tolist())
    print(f"✅ Retomando do ponto salvo: {len(processados)} registros já processados.")
except FileNotFoundError:
    resultado_parcial = pd.DataFrame()
    processados = set()
    print("🚀 Iniciando do zero.")

# === Lista final de resultados
resultados = resultado_parcial.to_dict("records")

# === Iterar com tqdm
batch = []
for i, row in tqdm(vagas_gpt.iterrows(), total=len(vagas_gpt)):
    uuid = row["uuid"]
    if uuid in processados:
        continue

    competencias = analisar_vaga_gpt(row["description"])
    competencias["uuid"] = uuid
    competencias["identifier"] = row["identifier"]
    competencias["title"] = row["title"]
    competencias["description"] = row["description"]
    batch.append(competencias)

    time.sleep(1)

    # Salvamento a cada 500 processadas
    if len(batch) >= 500:
        print(f"💾 Salvando {len(batch)} registros...")
        df_batch = pd.DataFrame(batch)
        resultado_parcial = pd.concat([resultado_parcial, df_batch], ignore_index=True)
        resultado_parcial.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
        processados.update(df_batch["uuid"].tolist())
        batch = []

# === Salvamento final
if batch:
    print(f"💾 Salvamento final de {len(batch)} registros restantes...")
    df_batch = pd.DataFrame(batch)
    resultado_parcial = pd.concat([resultado_parcial, df_batch], ignore_index=True)
    resultado_parcial.to_csv(caminho_saida, index=False, encoding="utf-8-sig")

print("✅ Processo concluído com sucesso.")
import pandas as pd
import json
from ast import literal_eval

# === 1. Carregar base antiga (já no formato final)
df_antiga = pd.read_json("base_ner_completa.jsonl", lines=True)

# === 2. Carregar nova base anotada via GPT
df_nova = pd.read_csv("resultado_gpt_parcial.csv", encoding="utf-8-sig")

# === 3. Converter colunas de listas em texto real
for col in ["Habilidades", "Conhecimentos", "Atitudes_Valores"]:
    df_nova[col] = df_nova[col].apply(lambda x: literal_eval(x) if isinstance(x, str) and x.startswith("[") else [])

# === 4. Gerar spans no formato {"start": int, "end": int, "label": str}
def gerar_spans(row):
    spans = []
    texto = row["description"].lower()

    for tipo, termos in {
        "HABILIDADE": row["Habilidades"],
        "CONHECIMENTO": row["Conhecimentos"],
        "ATITUDE_VALOR": row["Atitudes_Valores"]
    }.items():
        for termo in termos:
            termo = termo.strip().lower()
            if termo in texto:
                start = texto.find(termo)
                end = start + len(termo)
                spans.append({"start": start, "end": end, "label": tipo})
    return spans

df_nova["spans"] = df_nova.apply(gerar_spans, axis=1)
df_nova["text"] = df_nova["description"].str.lower()
df_nova_final = df_nova[["text", "spans"]].copy()
df_nova_final = df_nova_final[df_nova_final["spans"].map(len) > 0].reset_index(drop=True)

# === 5. Unir com a base antiga
df_combinada = pd.concat([df_antiga, df_nova_final], ignore_index=True)

# === 6. (Opcional) Remover duplicatas por texto + spans exatos
df_combinada["spans_json"] = df_combinada["spans"].apply(lambda x: json.dumps(sorted(x, key=lambda s: (s["start"], s["label"]))))
df_combinada = df_combinada.drop_duplicates(subset=["text", "spans_json"]).drop(columns=["spans_json"]).reset_index(drop=True)

# === 7. Salvar base combinada
df_combinada.to_json("base_ner_completa_retreino.jsonl", orient="records", lines=True, force_ascii=False)

print(f"✅ Base final salva com {len(df_combinada)} exemplos em: base_ner_completa_retreino.jsonl")


In [ ]:
vagas = pd.read_csv('vagas.csv', sep=";")
vagas_gpt = vagas.sample(5000)
habilidades_expandida = pd.read_csv('habilidades_expandidas.csv', sep=";")
# === Prompt de contexto ===
prompt_competencia_gpt = (
    "Você é um especialista em Processamento de Linguagem Natural (NLP) aplicado ao mercado de trabalho. "
    "Sua tarefa é analisar descrições de vagas de emprego e identificar, de forma estruturada, cinco categorias de informações relacionadas ao perfil profissional desejado:\n\n"
    "1. **Habilidades**: capacidades técnicas ou práticas que o candidato precisa demonstrar, como 'atendimento ao cliente', 'operar máquinas', 'vendas', 'comunicação assertiva';\n"
    "2. **Conhecimentos**: saberes técnicos, normativos ou teóricos necessários à função, como 'Excel', 'normas da Anvisa', 'legislação trabalhista'. **Não inclua níveis de escolaridade aqui**;\n"
    "3. **Atitudes e Valores**: comportamentos e disposições esperadas do profissional, como 'proatividade', 'trabalho em equipe', 'responsabilidade', 'ética', 'comprometimento';\n\n"
    "Ignore informações que **não** dizem respeito diretamente ao perfil do candidato, como:\n"
    "- Nome da empresa;\n"
    "- Localização da vaga;\n"
    "- Jornada de trabalho;\n"
    "- Tipo de contrato;\n"
    "- Benefícios;\n"
    "- Requisitos vagos como “experiência prévia” sem detalhamento da atividade.\n\n"
    "Retorne o resultado estruturado neste formato (sem explicações adicionais):\n"
    "Habilidades: [lista]\n"
    "Conhecimentos: [lista]\n"
    "Atitudes e Valores: [lista]\n"
        "- **Não inclua 'disponibilidade para viajar'.**\n\n"
    "Formato de resposta (sem explicações):\n"
    "Habilidades: [lista]\n"
    "Conhecimentos: [lista]\n"
    "Atitudes e Valores: [lista]\n\n"
    "Exemplo:\n"
    "\"Estamos em busca de um atendente para padaria. Será responsável por atendimento ao cliente, caixa, limpeza. Requer conhecimento em higiene alimentar. Buscamos uma pessoa proativa, simpática e com trabalho em equipe.\"\n"
    "Habilidades: ['atendimento ao cliente', 'operar o caixa', 'manter limpeza']\n"
    "Conhecimentos: ['higiene alimentar']\n"
    "Atitudes e Valores: ['proatividade', 'simpatia', 'trabalho em equipe']"
)

# === Função GPT ===
def analisar_vaga_gpt(descricao):
    if not isinstance(descricao, str) or descricao.strip() == "":
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade": [], "Certificacoes": []}
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt_competencia_gpt},
                {"role": "user", "content": descricao}
            ],
            temperature=0,
            max_tokens=800
        )
        content = response.choices[0].message.content

        resultado = {
            "Habilidades": [],
            "Conhecimentos": [],
            "Atitudes_Valores": []
        }
        for linha in content.split("\n"):
            if linha.lower().startswith("habilidades"):
                resultado["Habilidades"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif linha.lower().startswith("conhecimentos"):
                resultado["Conhecimentos"] = linha.split(":", 1)[-1].strip(" []").split(", ")
            elif "atitudes" in linha.lower():
                resultado["Atitudes_Valores"] = linha.split(":", 1)[-1].strip(" []").split(", ")
        return resultado
    except Exception as e:
        print(f"Erro: {e}")
        return {"Habilidades": [], "Conhecimentos": [], "Atitudes_Valores": [], "Escolaridade": [], "Certificacoes": []}


# === Carregar progresso anterior (se existir) ===
caminho_saida = "resultado_gpt_parcial2.csv"
try:
    resultado_parcial = pd.read_csv(caminho_saida, encoding="utf-8-sig")
    processados = set(resultado_parcial["uuid"].tolist())
    print(f"✅ Retomando do ponto salvo: {len(processados)} registros já processados.")
except FileNotFoundError:
    resultado_parcial = pd.DataFrame()
    processados = set()
    print("🚀 Iniciando do zero.")

# === Lista final de resultados
resultados = resultado_parcial.to_dict("records")

# === Iterar com tqdm
batch = []
for i, row in tqdm(vagas_gpt.iterrows(), total=len(vagas_gpt)):
    uuid = row["uuid"]
    if uuid in processados:
        continue

    competencias = analisar_vaga_gpt(row["description"])
    competencias["uuid"] = uuid
    competencias["identifier"] = row["identifier"]
    competencias["title"] = row["title"]
    competencias["description"] = row["description"]
    batch.append(competencias)

    time.sleep(1)

    # Salvamento a cada 500 processadas
    if len(batch) >= 500:
        print(f"💾 Salvando {len(batch)} registros...")
        df_batch = pd.DataFrame(batch)
        resultado_parcial = pd.concat([resultado_parcial, df_batch], ignore_index=True)
        resultado_parcial.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
        processados.update(df_batch["uuid"].tolist())
        batch = []

# === Salvamento final
if batch:
    print(f"💾 Salvamento final de {len(batch)} registros restantes...")
    df_batch = pd.DataFrame(batch)
    resultado_parcial = pd.concat([resultado_parcial, df_batch], ignore_index=True)
    resultado_parcial.to_csv(caminho_saida, index=False, encoding="utf-8-sig")

print("✅ Processo concluído com sucesso.")

In [ ]:
# === 1. Carregar base original no formato JSONL
df_antiga = pd.read_json("base_ner_completa_retreino.jsonl", lines=True)

# === 2. Carregar nova base anotada via GPT
df_nova = pd.read_csv("resultado_gpt_parcial2.csv", encoding="utf-8-sig")

# === 3. Converter colunas de string para listas reais
for col in ["Habilidades", "Conhecimentos", "Atitudes_Valores"]:
    df_nova[col] = df_nova[col].apply(lambda x: literal_eval(x) if isinstance(x, str) and x.startswith("[") else [])

# === 4. Função para gerar spans usando regex
def gerar_spans_regex(row):
    spans = []
    texto = row["description"]
    texto_lower = texto.lower()

    for tipo, termos in {
        "HABILIDADE": row["Habilidades"],
        "CONHECIMENTO": row["Conhecimentos"],
        "ATITUDE_VALOR": row["Atitudes_Valores"]
    }.items():
        for termo in termos:
            termo = termo.strip().lower()
            if not termo:
                continue
            termo_re = re.escape(termo)
            for match in re.finditer(rf'\b{termo_re}\b', texto_lower):
                spans.append({"start": match.start(), "end": match.end(), "label": tipo})
    return spans

# === 5. Gerar nova base com text + spans
df_nova["spans"] = df_nova.apply(gerar_spans_regex, axis=1)
df_nova["text"] = df_nova["description"]
df_nova_final = df_nova[["text", "spans"]].copy()
df_nova_final = df_nova_final[df_nova_final["spans"].map(len) > 0].reset_index(drop=True)

# === 6. Combinar com a base antiga e remover duplicatas
df_combinada = pd.concat([df_antiga, df_nova_final], ignore_index=True)
df_combinada["spans_json"] = df_combinada["spans"].apply(lambda x: json.dumps(sorted(x, key=lambda s: (s["start"], s["label"]))))
df_combinada = df_combinada.drop_duplicates(subset=["text", "spans_json"]).drop(columns=["spans_json"]).reset_index(drop=True)

# === 7. Salvar base final combinada
df_combinada.to_json("base_ner_completa_retreino2.jsonl", orient="records", lines=True, force_ascii=False)

print(f"✅ Base combinada salva com {len(df_combinada)} exemplos em 'base_ner_completa_retreino2.jsonl'")


In [ ]:
import pandas as pd

# Carregar o JSONL
df = pd.read_json("base_ner_completa_retreino2.jsonl", lines=True)

# Converter os spans para string para poder visualizar em CSV
df["spans"] = df["spans"].apply(str)

# Salvar como CSV
df.to_csv("base_ner_completa_retreino2.csv", index=False, encoding="utf-8-sig",  sep=";")

print("✅ Arquivo salvo como base_ner_completa_retreino2.csv")

In [ ]:
import pandas as pd
import json
from IPython.display import display, HTML

# === 1. Carregar base anotada
df = pd.read_json("base_ner_completa_retreino2.jsonl", lines=True)

# === 2. Função para destacar spans no texto
def destacar_spans(texto, spans):
    spans = sorted(spans, key=lambda x: x['start'])
    saida = ""
    ultimo_idx = 0
    cores = {
        "HABILIDADE": "#d4f4dd",
        "CONHECIMENTO": "#d0e7ff",
        "ATITUDE_VALOR": "#fbe4e6"
    }
    for span in spans:
        saida += texto[ultimo_idx:span['start']]
        cor = cores.get(span['label'], "#eee")
        saida += f"<span style='background-color:{cor}; padding:1px; border-radius:3px;'>{texto[span['start']:span['end']]}</span>"
        ultimo_idx = span['end']
    saida += texto[ultimo_idx:]
    return saida

# === 3. Exibir 5 exemplos aleatórios com destaques
amostra = df.sample(10, random_state=42)

for i, row in amostra.iterrows():
    texto_destacado = destacar_spans(row["text"], row["spans"])
    display(HTML(f"<h4>Exemplo {i+1}</h4><p>{texto_destacado}</p>"))

import json

input_path = "base_ner_completa_retreino2.jsonl"
output_path = "base_filtrada_com_spans.jsonl"

with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        data = json.loads(line)
        if "spans" in data and isinstance(data["spans"], list) and len(data["spans"]) > 0:
            fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print(f"✅ Base salva com apenas exemplos com spans em: {output_path}")


In [ ]:
import openai
import pandas as pd
import re
import json
import time
from tqdm import tqdm

# === Prompt único ===
prompt_geracao_completo = (
    "Você é um especialista em Recursos Humanos e Processamento de Linguagem Natural (NLP). "
    "Sua tarefa é gerar descrições realistas de vagas de emprego, e em seguida, analisar o texto gerado para identificar as competências envolvidas.\n\n"
    "Siga estes dois passos:\n"
    "1. Gere uma descrição curta e natural de vaga de emprego para qualquer profissão. "
    "A descrição deve mencionar:\n"
    "- pelo menos uma habilidade prática,\n"
    "- um conhecimento técnico ou normativo,\n"
    "- e uma atitude ou valor esperado do candidato.\n\n"
    "2. Logo após o texto da vaga, retorne as competências estruturadas no seguinte formato (sem explicações):\n"
    "Habilidades: [lista]\n"
    "Conhecimentos: [lista]\n"
    "Atitudes e Valores: [lista]\n\n"
    "**Exemplo de saída esperada:**\n"
    "Descrição: Estamos em busca de um analista de TI com domínio em Python, boa comunicação e experiência em elaboração de dashboards. Espera-se postura ética e capacidade de trabalhar em equipe.\n"
    "Habilidades: ['elaboração de dashboards']\n"
    "Conhecimentos: ['Python']\n"
    "Atitudes e Valores: ['comunicação', 'postura ética', 'trabalho em equipe']\n\n"
    "As competências extraídas das vagas devem ser IGUAIS ÀQUELAS NA DESCRIÇÃO, sem modificação de texto, como por exemplo 'detalhista' para 'detalhismo'. É preciso que sejam compatíveis"
    "Agora gere o exemplo real abaixo. Comece pela palavra 'Descrição:' e siga o formato."
)

# === Geração com GPT
def gerar_vaga_e_competencias():
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt_geracao_completo}],
            temperature=0.9,
            max_tokens=600
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Erro na geração: {e}")
        return None

# === Função para parsear a resposta do GPT
def limpar_lista(lista):
    return [termo.strip(" '\"") for termo in lista if termo.strip()]

def extrair_info(texto):
    try:
        partes = texto.split("\n")
        descricao = ""
        habilidades, conhecimentos, atitudes = [], [], []
        for linha in partes:
            if linha.lower().startswith("descrição"):
                descricao = linha.split(":", 1)[-1].strip()
            elif linha.lower().startswith("habilidades"):
                habilidades = limpar_lista(linha.split(":", 1)[-1].strip(" []").split(","))
            elif linha.lower().startswith("conhecimentos"):
                conhecimentos = limpar_lista(linha.split(":", 1)[-1].strip(" []").split(","))
            elif "atitudes" in linha.lower():
                atitudes = limpar_lista(linha.split(":", 1)[-1].strip(" []").split(","))
        return {
            "text": descricao,
            "Habilidades": habilidades,
            "Conhecimentos": conhecimentos,
            "Atitudes_Valores": atitudes
        }
    except Exception as e:
        print(f"Erro no parsing: {e}")
        return None


# === Função para gerar spans
def gerar_spans_regex(row):
    spans = []
    texto = row["text"]
    texto_lower = texto.lower()

    for tipo, termos in {
        "HABILIDADE": row["Habilidades"],
        "CONHECIMENTO": row["Conhecimentos"],
        "ATITUDE_VALOR": row["Atitudes_Valores"]
    }.items():
        for termo in termos:
            termo = termo.strip().lower()
            if not termo:
                continue
            termo_re = re.escape(termo)
            for match in re.finditer(rf'\b{termo_re}\b', texto_lower):
                spans.append({"start": match.start(), "end": match.end(), "label": tipo})
    return spans

# === Geração em lote
sintetica = []
num_exemplos = 3000

for _ in tqdm(range(num_exemplos)):
    resposta = gerar_vaga_e_competencias()
    if not resposta:
        continue
    dados = extrair_info(resposta)
    if not dados:
        continue
    spans = gerar_spans_regex(dados)
    if not spans:
        continue
    sintetica.append({"text": dados["text"], "spans": spans})
    time.sleep(1)

# === Criar e salvar DataFrame
df_sintetica = pd.DataFrame(sintetica)
df_sintetica.to_json("base_ner_sintetica_gpt.jsonl", orient="records", lines=True, force_ascii=False)

print(f"✅ Base salva com {len(df_sintetica)} exemplos.")

In [ ]:
# === 8. Carregar base anterior (para juntar)
df_antiga = pd.read_json("base_ner_completa_retreino.jsonl", lines=True)
df_completa = pd.concat([df_antiga, df_sintetica], ignore_index=True)

# === 9. Remover duplicatas
df_completa["spans_json"] = df_completa["spans"].apply(lambda x: json.dumps(sorted(x, key=lambda s: (s["start"], s["label"]))))
df_completa = df_completa.drop_duplicates(subset=["text", "spans_json"]).drop(columns=["spans_json"]).reset_index(drop=True)

# === 10. Salvar como JSONL
df_completa.to_json("base_ner_completa_retreino_sintetica.jsonl", orient="records", lines=True, force_ascii=False)

print(f"✅ Base final salva com {len(df_completa)} exemplos.")